# Resize and Move Images Notebook

This Jupyter notebook resizes images from the `needs_resizing` directory and moves them to the `static_screenshots` directory using the Python Imaging Library (PIL) and asyncio for asynchronous file handling.

## Requirements

- Python 3.x
- PIL (Pillow)
- aiofiles

In [ ]:
## Set the PATH Environment Variable in Your Notebook:
# You can modify the PATH environment variable within your notebook to prioritize the virtual environment.
# You can do this by adding the path of the Python executable of your virtual environment to the beginning of the PATH variable:
import sys
import os

from IPython import get_ipython

# Get the path of the Python executable
python_exec_path = sys.executable

# Set the PATH environment variable
os.environ['PATH'] = f"{os.path.dirname(python_exec_path)}:{os.environ['PATH']}"

# Now check the Python version with a shell command
!python --version

# Check the Python executable path
print(python_exec_path)

# Function to get the notebook directory
def get_notebook_dir():
    kernel_id = get_ipython().parent_header['msg_id']
    connection_file = get_ipython().config['IPKernelApp']['connection_file']
    kernel_dir = os.path.dirname(connection_file)

    for file in os.listdir(kernel_dir):
        if file.startswith(kernel_id) and file.endswith('.json'):
            with open(os.path.join(kernel_dir, file)) as f:
                notebook_path = json.load(f).get('notebook', {}).get('path', '')
                return os.path.dirname(notebook_path)
    return None

# Get the directory where the notebook is located
notebook_dir = get_notebook_dir()

if not notebook_dir:
    print("Could not determine the notebook directory.")

# Now install any dependent modules to your kernel
%pip install pillow aiofiles

In [ ]:
if not notebook_dir:
    # If notebook_dir is not defined, manually set it
    notebook_dir = os.path.expanduser("~/Git/urban-palm-tree/static_screenshots")

# Change the working directory to that location
os.chdir(notebook_dir)

# Verify the current working directory
print("Current working directory:", os.getcwd())

In [ ]:
import os
from PIL import Image
import asyncio
import aiofiles
import io

# Define directories
source_directory = "../needs_resizing"
destination_directory = "."
target_resolution=(480, 270)

# Ensure the destination directory exists
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

async def resize_and_move_image(image_path, destination_directory, target_resolution):
    try:
        # Load the image
        async with aiofiles.open(image_path, 'rb') as f:
            img_data = await f.read()
        
        img = Image.open(io.BytesIO(img_data))
        
        # Debug: Print image details before resizing
        print(f"Processing {image_path} with original size {img.size}")
        
        # Check if the image needs resizing
        if img.size != target_resolution:
            img = img.resize(target_resolution)
            # Debug: Print image details after resizing
            print(f"Resized image size: {img.size}")
        
        # Save the resized image to the destination directory
        filename = os.path.basename(image_path)
        destination_path = os.path.join(destination_directory, filename)
        
        img.save(destination_path, format="PNG")
        print(f"Saved resized image to {destination_path}")
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

async def main():
    tasks = []
    for filename in os.listdir(source_directory):
        if filename.lower().endswith(('png', 'jpg', 'jpeg')):
            image_path = os.path.join(source_directory, filename)
            tasks.append(resize_and_move_image(image_path, destination_directory, target_resolution))
    
    await asyncio.gather(*tasks)

# Run the main function
await main()

## Resize to 1920x1080
Will instead resize screenshots in /screenshots/jpg to 1920x1080

In [ ]:
import os
from PIL import Image

# Define the source and destination directories
source_directory = "../screenshots/jpg"
destination_directory = "../screenshots/jpg/resized"

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)

# Target resolution
target_resolution = (1920, 1080)

# Loop through all files in the source directory
for filename in os.listdir(source_directory):
    if filename.lower().endswith(('png', 'jpg', 'jpeg')):
        image_path = os.path.join(source_directory, filename)
        
        # Open and resize the image
        with Image.open(image_path) as img:
            resized_img = img.resize(target_resolution)
            destination_path = os.path.join(destination_directory, filename)
            resized_img.save(destination_path)  # Save the resized image

        print(f"Processed {filename}: resized to {target_resolution} and saved to {destination_path}")